here we run some baselines to understand the level of performance to beat
we use nested CV and grid search to get realistic performance estimates

In [1]:
from netCDF4 import Dataset
from collections import defaultdict, namedtuple
import datetime
import numpy as np
import seaborn as sns
sns.set()
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import math
import os
import csv
import dask.bag as db
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from sklearn.linear_model import Ridge
from sklearn.utils import shuffle
from sklearn import metrics
from hkfold import HKFold, train_test_split
from scipy.optimize import fmin_cg
from IPython.display import clear_output

In [2]:
dframe_path = 'data/cabauw/processed.csv.gz'

try:
    df = pd.read_csv(dframe_path, na_values='--')
except UnicodeDecodeError:
    df = pd.read_csv(dframe_path, na_values='--', compression='gzip')


df = df[(df.ustar > 0.1) & (abs(df.H) > 10) & (df.wind > 1)]
df = df[(df.ds != 201603) & (df.phi_m.notnull())]
df = df.sort_values(['ds', 'tt'])
df = df.dropna()

lets start with a baseline. data doesnt follow the functions given in the literature, so while I fix it we can change those functions to fit the data. given the definition of $\phi_m$ for $\xi>0$:

$$
\phi_m(\xi)=a+b\xi
$$

whose derivatives are trivial. for $\xi<0$ we have

$$
\phi_m(\xi)=(1-c^2\xi)^d
$$

where we square $c$ to make sure the base of the power is always positive. its derivatives are

$$
\frac{\partial \phi_m}{\partial c}=-2cd\xi(1-c^2\xi)^{d-1}
$$

$$
\frac{\partial \phi_m}{\partial d}=(1-c^2\xi)^d\ln(1-c^2\xi)
$$

considering the usual least squares with l2 regularization we have the loss function

$$
E=\frac{1}{N}\sum_i(\hat\phi_m(\xi_i)-\phi_m(\xi_i,p))^2+\frac{\lambda}{2}\sum_p p^2
$$

and its derivative with respect to the parameter $p$

$$
\frac{\partial E}{\partial p}=\frac{2}{N}\sum_i\frac{\partial}{\partial p}\phi_m(\xi_i,p)\cdot(\hat\phi_m(\xi_i)-\phi_m(\xi_i,p))+\lambda p
$$

In [4]:
class MOSTEstimator:
    ''' estimator for the universal functions in the monin-obukhov similarity theory
        implementing scikit's interface
        
        fitting is done by minimizing the L2 regularized squared error
        via conjugate gradient
    '''
    def __init__(self, regu=0.1):
        self.regu = regu
        self.a, self.b, self.c, self.d = (1, 4.8, np.sqrt(19.3), -0.25)
        
    def get_params(self, deep=True):
        return {'regu': self.regu}
    
    def set_params(self, regu):
        self.regu = regu
        return self

    @staticmethod
    def _compute_phi(zL, a, b, c, d):
        mask = zL >= 0
        yy = np.zeros(len(zL))
        yy[mask] = a + b * zL[mask]
        yy[~mask] = np.power(1 - c**2 * zL[~mask], d)
        assert all(np.isfinite(zL))
        assert all(np.isfinite(yy)), (a, b, c, d)
        return yy

    @staticmethod
    def _compute_phi_prime(zL, a, b, c, d):
        dpda, dpdb, dpdc, dpdd = np.zeros((4, len(zL)))

        pos, neg = zL >= 0, zL < 0

        dpda[pos] = 1
        dpdb[pos] = zL[pos]

        inner = 1 - c**2 * zL[neg]
        dpdc[neg] = -2 * zL[neg] * c * d * np.power(inner, d - 1)
        dpdd[neg] = np.log(inner) * np.power(inner, d)

        return dpda, dpdb, dpdc, dpdd
    
    @staticmethod
    def _fmin_target(params, xx, yy, regu):
        preds = MOSTEstimator._compute_phi(xx, *params)
        return np.mean((yy - preds)**2) + regu * sum(p**2 for p in params)

    @staticmethod
    def _fmingrad(params, xx, yy, regu):
        preds = MOSTEstimator._compute_phi(xx, *params)
        der = MOSTEstimator._compute_phi_prime(xx, *params)

        grads = [
            2 * np.mean((preds - yy) * parpr) + regu * par
            for par, parpr in zip(params, der)
        ]

        return np.array(grads)

    def fit(self, X, y):
        self.a, self.b, self.c, self.d = fmin_cg(
            self._fmin_target,
            (self.a, self.b, self.c, self.d),
            self._fmingrad,
            args=(X, y, self.regu),
            disp=False,
        )
        
        return self

    def predict(self, X):
        return self._compute_phi(X, self.a, self.b, self.c, self.d)

    def score(self, X, y):
        preds = self.predict(X)
        return metrics.mean_squared_error(y, preds)

In [3]:
class AttributeKFold:
    ''' k-fold cross validator splitting on a particular attribute
        so that all samples with a given value are either in the train or test set

        attribute value for each sample is given in the constructor, so that
        the attribute itself need not be in the features for the model
    '''
    def __init__(self, cv, attr):
        self.cv, self.attr = cv, attr

    def get_n_splits(self, *args, **kwargs):
        return self.cv.get_n_splits(*args, **kwargs)

    def split(self, X, y=None, groups=None):
        vals = self.attr.unique()
        for train_idx, test_idx in self.cv.split(vals):
            train_mask = self.attr.isin(vals[train_idx])
            test_mask = self.attr.isin(vals[test_idx])

            yield (
                np.argwhere(train_mask).reshape(-1),
                np.argwhere(test_mask).reshape(-1),
            )

In [135]:
outer_cv = AttributeKFold(KFold(10, shuffle=True), df.ds)
outer_train, outer_test = np.zeros((2, len(df)))
for outer_train_idx, outer_test_idx in outer_cv.split(df):
    
    outer_train[outer_train_idx] += 1
    outer_test[outer_test_idx] += 1
    
    inner_train, inner_test = np.zeros((2, len(outer_train_idx)))
    inner_cv = AttributeKFold(KFold(5, shuffle=True), df.iloc[outer_train_idx].ds)
    for inner_train_idx, inner_test_idx in inner_cv.split(df.iloc[outer_train_idx]):
        inner_train[inner_train_idx] += 1
        inner_test[inner_test_idx] += 1
    
    assert all(inner_train == 4)
    assert all(inner_test == 1)

assert all(outer_train == 9)
assert all(outer_test == 1)

In [4]:
def nested_cv(model, grid, features, target, n_jobs=-2):
    outer_cv = AttributeKFold(KFold(10, shuffle=True), df.ds)

    results = []
    for oi, (train_idx, test_idx) in enumerate(outer_cv.split(df.ds)):
        # get training and test samples
        train_x, train_y = df.iloc[train_idx][features], df.iloc[train_idx][target]
        test_x, test_y = df.iloc[test_idx][features], df.iloc[test_idx][target]

        # normalize to 0 meand and unit std
        mean_x, std_x = train_x.mean(), train_x.std()
        train_x = (train_x - mean_x) / std_x
        test_x = (test_x - mean_x) / std_x

        mean_y, std_y = train_y.mean(), train_y.std()
        train_y = (train_y - mean_y) / std_y
        test_y = (test_y - mean_y) / std_y

        # grid search for best params
        inner_cv = AttributeKFold(KFold(5, shuffle=True), df.iloc[train_idx].ds)
        gs = GridSearchCV(
            model, grid, n_jobs=n_jobs, cv=inner_cv,
            scoring='neg_mean_squared_error',
            verbose=2,
        )
        gs.fit(train_x, train_y)
        
        # evaluate on test data
        y_pred = gs.best_estimator_.predict(test_x)
        results.append((
            metrics.explained_variance_score(test_y, y_pred),
            metrics.mean_absolute_error(test_y, y_pred),
            metrics.mean_squared_error(test_y, y_pred),
            metrics.median_absolute_error(test_y, y_pred),
            metrics.r2_score(test_y, y_pred),
            np.mean(np.abs((test_y - y_pred) / test_y)) * 100,
        ))
        
    return pd.DataFrame(results, columns=[
        'explained_variance_score',
        'mean_absolute_error',
        'mean_squared_error',
        'median_absolute_error',
        'r2_score',
        'mean_percent_error'
    ]), test_x, test_y, y_pred

In [5]:
def plot_preds(ypred, ytrue):
    minn = max(min(ypred), min(ytrue))
    maxx = min(max(ypred), max(ytrue))
    
    plt.scatter(ytrue, ypred, s=2)
    plt.plot([minn, maxx], [minn, maxx], 'r--')
    plt.xlabel('True')
    plt.ylabel('Predicted')
    
    plt.show()

In [ ]:
most_res, testx, testy, ypred = nested_cv(MOSTEstimator(), {
    'regu': [0, 1e-2, 1e-1, 1, 1e1, 1e2]
}, 'zL', 'phi_m')
plot_preds(ypred, testy)
most_res.describe()

In [6]:
features = [
    'dewpoint', 'spec_hum', 'rel_hum', 'press', 'rain',
    'air_dens', 'wind', 'temp', 'virtual_temp', 'soil_temp', 'z',
]

target = 'phi_m'

In [7]:
knn_res, testx, testy, ypred = nested_cv(KNeighborsRegressor(),  {
    'n_neighbors': [1, 5, 10],
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
}, features, target)
plot_preds(ypred, testy)
knn_res

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] n_neighbors=1, p=1, weights=uniform .............................
[CV] n_neighbors=1, p=1, weights=uniform .............................
[CV] n_neighbors=1, p=1, weights=uniform .............................
[CV] n_neighbors=1, p=1, weights=uniform .............................
[CV] n_neighbors=1, p=1, weights=uniform .............................
[CV] n_neighbors=1, p=1, weights=distance ............................
[CV] n_neighbors=1, p=1, weights=distance ............................


KeyboardInterrupt: 

In [23]:
rf_res, testx, testy, ypred = nested_cv(RandomForestRegressor(),  {
    'n_estimators': [100],
    'max_features': [3, 5, 11],
    'min_samples_split': [10, 100, 500, 1000],
    'min_samples_leaf': [10, 100, 500, 1000],
    'n_jobs': [7],
}, features, target, n_jobs=1)
plot_preds(ypred, testy)
rf_res

Fitting 5 folds for each of 48 candidates, totalling 240 fits


KeyboardInterrupt: 

In [ ]:
lrres, testx, testy, ypred = nested_cv(Ridge(), {
    'alpha': [0, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]
}, features, target)
plot_preds(ypred, testy)
lrres.describe()